In [2]:
import tensorflow as tf
import numpy as np

# Settings

In [3]:
TRAIN_ID = range(1,11)         # Raw file format : Sample0001.zip - Sample0470.zip
VALIDATION_ID = range(471,481)  # Raw file format : Sample0471.zip - Sample0700.zip
TEST_ID = range(701,711)        # Raw file format : Sample0701.zip - Sample0941.zip
# TRAIN_LIST = range(1,471)         # Raw file format : Sample0001.zip - Sample0470.zip
# VALIDATION_LIST = range(471,701)  # Raw file format : Sample0471.zip - Sample0700.zip
# TEST_LIST = range(701,941)        # Raw file format : Sample0701.zip - Sample0941.zip

IMAGE_SIZE = (80, 80, 3)

In [4]:
TFRecord_DATA_PATH = './tf-data/'

In [5]:
FRAMES_PER_CLIP = 5
MAX_FRAMES = 2000
CLIPS_PER_VIDEO = MAX_FRAMES / FRAMES_PER_CLIP

'''Self-defined gesture labels'''
NO_GESTURE = 21
EMPTY_PADDING = 22

In [ ]:
'''Training parameters'''
NUM_EPOCHS = 1
NUM_READ_THREADS = 4


# input Pipeline

In [ ]:
def preprocessing_op(image_op):
    '''most preprocess should be done while making the .tfrecords files?'''
    with tf.name_scope('preprocessing'):
        # Reshape serialized image
        image_op = tf.reshape(image_op, IMAGE_SIZE)
        # Integer to float
        image_op = tf.to_float(image_op)
        # Normalize (Zero-mean unit-variance) on single image
        image_op = tf.image.per_image_standardization(image_op)
        
        return image_op
    
def read_and_decode(filename_queue):
    readerOptions = tf.python_io.TFRecordOptions(compression_type=tf.python_io.TFRecordCompressionType.GZIP)
    reader = tf.TFRecordReader(options=readerOptions)
    _, serialized_example = reader.read(filename_queue)
    
    with tf.name_scope('TFRecordDecoding'):
        context_encoded, features_encoded = tf.parse_single_sequence_example(
            serialized_example,
            context={'sample_id': tf.FixedLenFeature([], dtype=tf.int64)},
            feature={'rgb':tf.FixedLenSequenceFeature([],dtype=tf.string), # TODO: check string or bytes
                     'label':tf.FixedLenSequenceFeature([], dtype=tf.string)} # TODO: check string or bytes
        )
        seq_rgb = tf.decode_raw(features_encoded['rgb'], tf.int64)
        seq_label = tf.decode_raw(features_encoded['label'], tf.int64)
        # apply preprocessing to single image using map_fn
        seq_rgb = tf.map_fn(lambda x: preprocessing_op(x), elems = seq_rgb, dtype=tf.float32, back_prop=False)
        #TODO : back_pop = false??
        
        return [seq_rgb, seq_label]

def input_pipeline(filenames):
    with tf.name_scope('input_pipeline'):
        # Create a input file queue
        filename_queue = tf.train.string_input_producer(filenames, num_epochs=NUM_EPOCHS, shuffle=True)
        
        # Read data from .tfreords files and decode to a list of samples (Using threads)
        samples = [read_and_decode(filename_queue) for _ in range(NUM_READ_THREADS)]
        
        # Create batches
        # batch_join is used for N threads, can use batch_join_shuffle too
        batch_rgb, batch_labels = tf.train.batch_join(samples, 
                                                      batch_size = BATCH_SIZE,
                                                      capacity = QUEUE_CAPACITY,
                                                      enqueue_many= False, 
                                                      dynamic_pad = False, # make true if each data has different size
                                                      name = 'batch_join')
        return batch_rgb, batch_labels


# Training